<a href="https://colab.research.google.com/github/vedantdave77/Machine_Learning_feature_selection/blob/master/CFS_DT_KNN_SVM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Objective: 
#### Train the machine learning algorithm to observe the effect of the feature selection method (CFS- Correlation Feature Selection)  and improvement in Accuracy. 
The algorithms we use in project are:
1)Decision Tree  ||
2) KNN (Unsupervised)  ||
3) SVM  (Supervised)

## Flow of ML Program:
Read Data ----> Seperate in X and Y ----> Data Preprocessing ----> Split in Train and test set ----> CFS Feature selectin ----> No of feature (consideration decision) ----> Train Decision Tree / KNN / SVM ----> Make prediction ----> Accuracy score calculation ----> 10 Fold Cross Validation ----> Measure F1 score and Result comparison. 

In [0]:
#To import files
from google.colab import files
uploaded = files.upload()

Saving Gear1.xlsx to Gear1.xlsx


In [0]:
#Feasture selection Library install path

#@article{li2018feature,
 # title={Feature selection: A data perspective},
  #author={Li, Jundong and Cheng, Kewei and Wang, Suhang and Morstatter, Fred and Trevino, Robert P and Tang, Jiliang and Liu, Huan},
  #journal={ACM Computing Surveys (CSUR)},
  #volume={50},
  #number={6},
  #pages={94},
  #year={2018},
  #publisher={ACM}
#}

!pip install git+https://github.com/jundongl/scikit-feature.git

  Cloning https://github.com/jundongl/scikit-feature.git to /tmp/pip-req-build-i274_pfr
  Running command git clone -q https://github.com/jundongl/scikit-feature.git /tmp/pip-req-build-i274_pfr
  Created wheel for skfeature: filename=skfeature-1.0.0-cp36-none-any.whl size=61496 sha256=dd3a1ac6c03bc5a9259ed2806dca2bb029b671aef63cf275912e69fd2288d8e5
  Stored in directory: /tmp/pip-ephem-wheel-cache-jiqrohmv/wheels/5f/b4/c0/c8e13a8fbfaed3dd4b778d6788a3f9044d02b97a820a37f435
Successfully built skfeature


### Import Libraries, and required packages  

In [0]:
from google.colab import files
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn
from sklearn.model_selection import cross_validate
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report
from sklearn import tree
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import MinMaxScaler
from sklearn.utils import shuffle
from sklearn import metrics
from skfeature.utility import construct_W
from skfeature.function.statistical_based.CFS import cfs 
from skfeature.function.statistical_based.CFS import merit_calculation 
from sklearn import svm
import warnings
warnings.simplefilter('ignore')


#Read data in X and Y
df=pd.read_excel('Gear1.xlsx')
df_shuffle=shuffle(df);
df=df_shuffle;
y = df['Class']
x1 = df.drop(['Class'],axis=1)


#Data preprocessing
x=x1.loc[:,(df!=0).any(axis=0)];
scaler=MinMaxScaler(copy=True,feature_range=(0,1));
x_scaled=scaler.fit_transform(x)
x=x_scaled


# use train and test split
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2)


#CFS feature Selection
scores=cfs(x_train,y_train)
merit=merit_calculation(x_train,y_train)


#Number of features
no_features=scores.size


# check classification accuracy of classification tree
dt = tree.DecisionTreeClassifier()
dt = dt.fit(x_train[:, scores[0:no_features]], y_train)
y_pred = dt.predict(x_test[:, scores[0:no_features]])
print(metrics.accuracy_score(y_test, y_pred))

# check classification accuracy of KNN
knn = KNeighborsClassifier(n_neighbors=1)
knn=knn.fit(x_train[:, scores[0:no_features]], y_train)
y2_pred = knn.predict(x_test[:, scores[0:no_features]])
print(metrics.accuracy_score(y_test, y2_pred))


#check classification accuracy of SVM
clf = svm.SVC(C=35000, gamma=1000,kernel='rbf',random_state=0)
clf = clf.fit(x_train[:, scores[0:no_features]], y_train)
y3_pred=clf.predict(x_test[:, scores[0:no_features]])
print(metrics.accuracy_score(y_test, y3_pred))


# 10-fold cross-validation for decision tree
f_measure = cross_val_score(dt,x,y,cv=10,scoring='f1_weighted')
accuracy = cross_val_score(dt, x, y, cv=10, scoring='accuracy')


# 10-fold cross-validation for KNN
f_measure_k = cross_val_score(knn,x,y,cv=10,scoring='f1_weighted')
accuracy_k = cross_val_score(knn, x, y, cv=10, scoring='accuracy')


# 10-fold cross-validation for SVM
f_measure_svm = cross_val_score(clf,x,y,cv=10,scoring='f1_weighted')
accuracy_svm  = cross_val_score(clf,x,y,cv=10, scoring='accuracy')




#Create Table1
columns=np.transpose([f_measure,accuracy])
df=pd.DataFrame(columns,columns=['f_measure','accuracy'])
col_1=np.transpose(f_measure)
col_2=np.transpose(accuracy)
print('f_measure_DT',col_1)
print('accuracy_DT',col_2)


#Create Table2
columnsk=np.transpose([f_measure_k,accuracy_k])
dfk=pd.DataFrame(columnsk,columns=['f_measure','accuracy'])
col_3=np.transpose(f_measure_k)
col_4=np.transpose(accuracy_k)
print('f_measure_KNN',col_3)
print('accuracy_KNN',col_4)


#Create Table3
columnsvm=np.transpose([f_measure_svm,accuracy_svm])
dfsvm=pd.DataFrame(columnsvm,columns=['f_measure','accuracy'])
col_5=np.transpose(f_measure_svm)
col_6=np.transpose(accuracy_svm)
print('f_measure_SVM',col_5)
print('accuracy_SVM',col_6)


#export file1
file='UDFS-DT'+'.xlsx'
exportfile=df.to_excel(file,index= None,header=True)
files.download(file)


#export file2
file2='UDFS-KNN'+'.xlsx'
exportfile2=dfk.to_excel(file2,index=None,header=True)
files.download(file2)

#export file3
file3='UDFS-SVM'+'.xlsx'
exportfile3=dfsvm.to_excel(file3,index=None,header=True)
files.download(file3)

0.8561368209255533
0.9054325955734407
0.8631790744466801
f_measure_DT [0.84149094 0.79030259 0.82263039 0.85347701 0.80805176 0.82357068
 0.83081094 0.80606856 0.83175472 0.825247  ]
accuracy_DT [0.84931507 0.8011811  0.81818182 0.83697813 0.8247012  0.80923695
 0.83943089 0.82304527 0.83783784 0.8308977 ]
f_measure_KNN [0.8701576  0.84801092 0.85933266 0.86872284 0.89025635 0.84475205
 0.87459837 0.84840055 0.8663358  0.8659279 ]
accuracy_KNN [0.87084149 0.8484252  0.86166008 0.86878728 0.89243028 0.84538153
 0.87398374 0.84979424 0.86902287 0.86847599]
f_measure_SVM [0.78491772 0.76250548 0.76886221 0.77833958 0.78493619 0.76100612
 0.79771488 0.78921746 0.81395977 0.80773516]
accuracy_SVM [0.72407045 0.69488189 0.7055336  0.71172962 0.7250996  0.69277108
 0.7398374  0.73045267 0.76299376 0.75365344]


MessageError: ignored

### Observation:
Accuracy and F1 Score are depend on the parameter tuning , but with more restricted tuning we found the drop in accuracy in prediction which shows the Overfitting of model. 